# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [23]:
from selenium import webdriver # CRIAR O NAVEGADOR
from selenium.webdriver.common.by import By # LOCALIZAR ELEMENTOS 
from selenium.webdriver.common.keys import Keys # PERMITE CLICAR TECLAS 

navegador = webdriver.Chrome()
# PASSO 1 ENTRAR NO GOOGLE
navegador.get('https://www.google.com/')
# PASSO 2 PESQUISAR A COTAÇÃO DO DÓLAR
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# PASSO 3 PEGAR A COTAÇÃ DO DOLAR
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)
navegador.get('https://www.google.com/')
# PASSO 2 PESQUISAR A COTAÇÃO DO EURO
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)
# PASSO 5 PEGAR A COTAÇÃO DO OURO
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
print(cotacao_ouro)




5.0918
5.310039639
297,10


In [24]:
!pip install selenium

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [25]:
# PASSO 6 ATUALIZAR A MINHA BASE DE DADOS COM AS NOVAS COTAÇÕES
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [26]:
# ATUALIZAR A COTACAO DE ACORDO COM A MOEDA CORRESPONDENTE
# AS LINHAS AONDE A COLUNA "MOEDA" = "DOLAR"
# DOLAR
tabela.loc[tabela["Moeda"] == "Dólar", 'Cotação'] = float(cotacao_dolar)

# EURO
tabela.loc[tabela["Moeda"] == "Euro", 'Cotação'] = float(cotacao_euro)

# OURO
tabela.loc[tabela["Moeda"] == "Ouro", 'Cotação'] = float(cotacao_ouro.replace(",", "."))
# MOSTRAR
display(tabela)
# ATUALIZAR PRECO DE COMPRA = PRECO ORIGINAL * COTACAO
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# ATUALIZAR PRECO DE VENDA = PRECO DE COMPRA * MARGEM
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.09180,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.31004,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.09180,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.09180,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.31004,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.09180,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,297.10000,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.09180,5091.749082,1.40,7128.448715
1,Carro Renault,4500.00,Euro,5.31004,23895.178375,2.00,47790.356751
2,Notebook Dell,899.99,Dólar,5.09180,4582.569082,1.70,7790.367439
3,IPhone,799.00,Dólar,5.09180,4068.348200,1.70,6916.191940
4,Carro Fiat,3000.00,Euro,5.31004,15930.118917,1.90,30267.225942
5,Celular Xiaomi,480.48,Dólar,5.09180,2446.508064,2.00,4893.016128
6,Joia 20g,20.00,Ouro,297.10000,5942.000000,1.15,6833.300000


### Agora vamos exportar a nova base de preços atualizada

In [27]:
tabela.to_excel("Produtos_novos.xlsx", index=False)
navegador.quit()